In [1]:
import sys
sys.path.append("../")
import numpy as np
import time
import pandas as pd
import pickle
import math
from typing import Tuple


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

from scipy.sparse import hstack

ANALYSIS_POSTFIX = "mined_sudden_2024-08-26"

experiment_config = {
    "RS" : 42,
    "ANALYSIS_POSTFIX": ANALYSIS_POSTFIX,
    "FEATURE_MODE" : "CODE_MODEL", # CODE_MODEL
}

In [2]:
def step_two(experiment_config, 
             X_train,
             y_train,
             model,
             X_val=None,
             y_val=None,
             save=False): 
    
    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]
    
    training_start_time = time.time()
    if model=="lr":
        reg = LinearRegression().fit(X_train, y_train)
    elif model =="svm": 
        reg = SVR().fit(X_train, y_train)
    elif model=="rf":
        reg = RandomForestRegressor.fit(X_train, y_train)
    elif model=="lgbm":
        reg = LGBMRegressor(max_depth=10, silent=True)
        reg.fit(X=X_train, y=y_train)
    elif model=="catboost":
        reg = CatBoostRegressor()
        reg.fit(X=X_train, y=y_train)
    training_end_time = time.time()
    time_training = training_end_time - training_start_time

    
    if save:
        with open(f'./models/reg_{model}_{ANALYSIS_POSTFIX}.pkl','wb') as f:
            pickle.dump(reg, f)
        return f'./models/reg_{model}_{ANALYSIS_POSTFIX}.pkl'
    
    else:
        inference_start_time = time.time()
        y_pred = reg.predict(X_val)
        inference_end_time = time.time()
        time_inference = inference_end_time - inference_start_time

        y_pred[y_pred<0] = 0
        mae = mean_absolute_error(y_true=y_val, y_pred=y_pred)
        rmse = math.sqrt(mean_squared_error(y_true=y_val, y_pred=y_pred))
        return {"pred": y_pred, "mae": mae, "rmse": rmse, "time_training" : time_training, "time_inference" : time_inference}
    

def cv_step_2(experiment_config:dict, cv_df:pd.DataFrame) -> Tuple:

    t_models = ["lr", "svm", "lgbm", "catboost"]

    results = {}

    FEATURE_MODE = experiment_config["FEATURE_MODE"]

    for test_fold in range(cv_df.fold.max()+1):
        print(test_fold)

        # Prepare the input data
        vectorizer = TfidfVectorizer()
        X_train_tfidf = vectorizer.fit_transform(cv_df.loc[cv_df.fold!=test_fold, "input_sequence"])

        if FEATURE_MODE=="CODE_MODEL":
            X_train_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold!=test_fold, "model_set"], sparse=True).sparse.to_coo().tocsr()
            X_train = hstack([X_train_column_sparse, X_train_tfidf])
        elif FEATURE_MODE=="CODE":
            X_train = X_train_tfidf
            
        y_train = cv_df.loc[cv_df.fold!=test_fold, "rouge"]
        
        X_val_tfidf = vectorizer.transform(cv_df.loc[cv_df.fold==test_fold, "input_sequence"])
        if FEATURE_MODE=="CODE_MODEL":
            X_val_column_sparse = pd.get_dummies(cv_df.loc[cv_df.fold==test_fold, "model_set"], sparse=True).sparse.to_coo().tocsr()
            X_val = hstack([X_val_column_sparse, X_val_tfidf])
        elif FEATURE_MODE=="CODE":
            X_val = X_val_tfidf
            
        y_val = cv_df.loc[cv_df.fold==test_fold, "rouge"]

        results[test_fold] = {}
        for model in t_models:
            print(model)
            preds_df = step_two(experiment_config=experiment_config,
                                X_train=X_train,
                                y_train=y_train,
                                X_val=X_val,
                                y_val=y_val,
                                model=model)
            cv_df.loc[cv_df.fold==test_fold, f"{model}_perf_hat"] = preds_df["pred"]
            results[test_fold][model] = preds_df

    cv_df = cv_df.reset_index(drop=True)

    return cv_df

def full_step_2(cv_df:pd.DataFrame,
                experiment_config:dict) -> None:
    
    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]
    # TRAIN ON ALL PREDICTIONS AT ONCE

    t_models = ["lr", "svm", "lgbm", "catboost"]
    FEATURE_MODE = experiment_config["FEATURE_MODE"]

    # Prepare the input data
    vectorizer = TfidfVectorizer()
    X_train_tfidf = vectorizer.fit_transform(cv_df.loc[cv_df.model_set!="ensemble", "input_sequence"])
    if FEATURE_MODE=="CODE_MODEL":
        X_train_column_sparse = pd.get_dummies(cv_df.loc[cv_df.model_set!="ensemble", "model_set"], sparse=True).sparse.to_coo().tocsr()
        X_train = hstack([X_train_column_sparse, X_train_tfidf])
    elif FEATURE_MODE=="CODE":
        X_train = X_train_tfidf
        
    y_train = cv_df.loc[cv_df.model_set!="ensemble", "rouge"]
        
    with open(f"./models/vectorizer_{ANALYSIS_POSTFIX}.pkl", "wb") as file:
        pickle.dump(vectorizer, file, protocol=pickle.HIGHEST_PROTOCOL) 
        
    for model in t_models:
        print(model)
        preds_df = step_two(experiment_config=experiment_config,
                            X_train=X_train,
                            y_train=y_train,
                            model=model,
                            save=True)
        
def pred_perf(experiment_config,
              X,
              model): 

    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]

    with open(f'./models/reg_{model}_{ANALYSIS_POSTFIX}.pkl','rb') as f:
            reg = pickle.load(f)

    y_pred = reg.predict(X)
    y_pred[y_pred<0] = 0
    return y_pred

def meta_predict(experiment_config:dict, 
                 test_df: pd.DataFrame,
                 base_models_names: list,
                 t_models:list = ["svm", "catboost"]) -> pd.DataFrame:

    ANALYSIS_POSTFIX = experiment_config["ANALYSIS_POSTFIX"]
    FEATURE_MODE = experiment_config["FEATURE_MODE"]
    
    for model_i, model_set in enumerate(base_models_names):

        set_df = test_df.copy()
        set_df["model_set"] = model_set
        # Prepare the input data
        with open(f"./models/vectorizer_{ANALYSIS_POSTFIX}.pkl", "rb") as file:
            vectorizer = pickle.load(file)

        if model_i==0:
            meta_preds_df = set_df.copy()
        else: 
            meta_preds_df = pd.concat([meta_preds_df, set_df])
            
    X_test_tfidf = vectorizer.transform(meta_preds_df.loc[:, "input_sequence"])
    if FEATURE_MODE=="CODE_MODEL":
        X_test_column_sparse = pd.get_dummies(meta_preds_df.loc[:, "model_set"], sparse=True).sparse.to_coo().tocsr()
        X_test = hstack([X_test_column_sparse, X_test_tfidf])
    elif FEATURE_MODE=="CODE":
        X_test = X_test_tfidf

    for model in t_models:
        print(model)
        meta_preds_df[f"{model}_preds"] = pred_perf(experiment_config=experiment_config, 
                                                    X=X_test,
                                                    model=model)

    meta_preds_df = meta_preds_df.reset_index(drop=True)
    return meta_preds_df

In [3]:
with open(f"../ensemble_learning/reports/results/{ANALYSIS_POSTFIX}/cv_results.pickle", "rb") as handle:
    cv_predictions = pickle.load(handle)

with open(f"../ensemble_learning/reports/results/{ANALYSIS_POSTFIX}/test_results.pickle", "rb") as handle:
    test_predictions = pickle.load(handle)


In [4]:
COLUMNS_TEST = ['question_id', 'parent_answer_post_id', 'prob', 'input_sequence',
       'output_sequence', 'id', 'snippet_len', 'intent_len', 'snippet_token_n',
       'intent_token_n', 'cluster', 'input_ids', 'attention_mask', 'labels',
       'prediction', 'rouge', 'model_set']

COLUMNS_CV = COLUMNS_TEST.copy()
COLUMNS_CV.append("fold")

#### Preprocessing

In [5]:
cv_predictions = cv_predictions.loc[cv_predictions.model_set!="ensemble", COLUMNS_CV]
test_predictions = test_predictions.loc[cv_predictions.model_set!="ensemble", COLUMNS_TEST]

# Code Only

We have 9 base lerner settings models that we compare learning of 1, splitting to two meta models,  all together. 

In [6]:
MODELS_LIST = [0, 1, 2, 5, 10, 'cluster_[1]', 'cluster_[4]', 'cluster_[3]', 'cluster_[0, 1, 4]']
MODE = ["ONE-BY-ONE", "TWO-MODELS", "ALL"]

In [8]:
results_cv_df = pd.DataFrame()

t_models = ["lr", "svm", "lgbm", "catboost"]

temp_df =  cv_predictions.copy()
temp_df = cv_step_2(experiment_config=experiment_config,
            cv_df=temp_df)

for model_meta in t_models:
    for cluster in sorted(temp_df.cluster.unique()):

        print(cluster)
        cluster_temp_df = temp_df.loc[temp_df.cluster==cluster, :]

        mae = mean_absolute_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                    y_pred=cluster_temp_df.loc[:, f"{model_meta}_perf_hat"])
        
        rmse = math.sqrt(mean_squared_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                    y_pred=cluster_temp_df.loc[:, f"{model_meta}_perf_hat"]))

        fold_mae = []
        fold_rmse = []

        for fold in temp_df.fold.unique():
            fold_temp_df = cluster_temp_df.loc[cluster_temp_df.fold==fold, :]

            mae = mean_absolute_error(y_true=fold_temp_df.loc[:, "rouge"],
                                        y_pred=fold_temp_df.loc[:, f"{model_meta}_perf_hat"])
            
            rmse = math.sqrt(mean_squared_error(y_true=fold_temp_df.loc[:, "rouge"],
                                        y_pred=fold_temp_df.loc[:, f"{model_meta}_perf_hat"]))
            
            fold_mae.append(mae)
            fold_rmse.append(rmse)
        
        t_res = pd.DataFrame(data={"model_base": "all",
                                    "model_meta": model_meta,
                                    "cluster": cluster,
                                     "rmse_mean": rmse,
                                     "rmse_std": np.array(fold_rmse).std(),
                                     "mae_mean": mae,
                                     "mae_std" : np.array(fold_mae).std()}, index=[0])


        results_cv_df = pd.concat([results_cv_df, t_res], axis=0)
    


for model_meta in t_models:


    mae = mean_absolute_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model_meta}_perf_hat"])
    
    rmse = math.sqrt(mean_squared_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model_meta}_perf_hat"]))
    
    t_res = pd.DataFrame(data={"model_base": "all", "model_meta": model_meta, "cluster": "full", "rmse": rmse, "mae": mae,}, index=[0])


    results_cv_df = pd.concat([results_cv_df, t_res], axis=0)

results_cv_df = results_cv_df.sort_values(["model_meta", "cluster"])


0
lr
svm
lgbm
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 8.172387 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 13269
[LightGBM] [Info] Number of data points in the train set: 41994, number of used features: 1041
[LightGBM] [Info] Start training from score 0.229969
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -in

In [9]:
print("MAE")
display(results_cv_df.groupby(["model_meta", "cluster"], as_index=False)["mae"].describe())

MAE


,model_meta,cluster,count,mean,std,min,25%,50%,75%,max
0,catboost,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,catboost,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,catboost,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,catboost,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,catboost,4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,catboost,full,1.0,0.126168,NaN,0.126168,0.126168,0.126168,0.126168,0.126168
6,lgbm,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,lgbm,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,lgbm,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,lgbm,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
print("RMSE")
display(results_cv_df.groupby(["model_meta", "cluster"], as_index=False)["rmse"].describe())

RMSE


,model_meta,cluster,count,mean,std,min,25%,50%,75%,max
0,catboost,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,catboost,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,catboost,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,catboost,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,catboost,4,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,catboost,full,1.0,0.157777,NaN,0.157777,0.157777,0.157777,0.157777,0.157777
6,lgbm,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,lgbm,1,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,lgbm,2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,lgbm,3,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
results_test_df = pd.DataFrame()

t_models = ["lr", "svm", "lgbm", "catboost"]

cv_temp_df =  cv_predictions.copy()
temp_df =  test_predictions.copy()
full_step_2(experiment_config=experiment_config,
                        cv_df=cv_temp_df)
temp_df = meta_predict(experiment_config=experiment_config,
                        test_df=temp_df,
                        base_models_names=MODELS_LIST,
                        t_models=t_models)

for model_meta in t_models:
    for cluster in sorted(temp_df.cluster.unique()):

        print(cluster)
        cluster_temp_df = temp_df.loc[temp_df.cluster==cluster, :]


        mae = mean_absolute_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                    y_pred=cluster_temp_df.loc[:, f"{model_meta}_preds"])
        
        rmse = math.sqrt(mean_squared_error(y_true=cluster_temp_df.loc[:, "rouge"],
                                    y_pred=cluster_temp_df.loc[:, f"{model_meta}_preds"]))
        
        t_res = pd.DataFrame(data={"model_base": "all", "model_meta": model_meta, "cluster": cluster, "rmse": rmse, "mae": mae}, index=[0])

        results_test_df = pd.concat([results_test_df, t_res], axis=0)
    
for model_meta in t_models:


    mae = mean_absolute_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model_meta}_preds"])
    
    rmse = math.sqrt(mean_squared_error(y_true=temp_df.loc[:, "rouge"],
                                    y_pred=temp_df.loc[:, f"{model_meta}_preds"]))
    
    t_res = pd.DataFrame(data={"model_base": "all", "model_meta": model_meta, "cluster": "full", "rmse": rmse, "mae": mae,}, index=[0])

    results_test_df = pd.concat([results_test_df, t_res], axis=0)


results_test_df = results_test_df.sort_values(["model_meta", "cluster"])


lr
svm
lgbm
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Unknown parameter: silent
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 10.378645 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 19322
[LightGBM] [Info] Number of data points in the train set: 63000, number of used features: 1362
[LightGBM] [Info] Start training from score 0.229955
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

In [12]:
print("MAE")
display(results_test_df.groupby(["model_meta", "cluster"], as_index=False)["mae"].describe())

MAE


,model_meta,cluster,count,mean,std,min,25%,50%,75%,max
0,catboost,0,1.0,0.124023,NaN,0.124023,0.124023,0.124023,0.124023,0.124023
1,catboost,1,1.0,0.132703,NaN,0.132703,0.132703,0.132703,0.132703,0.132703
2,catboost,2,1.0,0.137622,NaN,0.137622,0.137622,0.137622,0.137622,0.137622
3,catboost,3,1.0,0.140726,NaN,0.140726,0.140726,0.140726,0.140726,0.140726
4,catboost,4,1.0,0.140509,NaN,0.140509,0.140509,0.140509,0.140509,0.140509
5,catboost,full,1.0,0.140109,NaN,0.140109,0.140109,0.140109,0.140109,0.140109
6,lgbm,0,1.0,0.125586,NaN,0.125586,0.125586,0.125586,0.125586,0.125586
7,lgbm,1,1.0,0.125864,NaN,0.125864,0.125864,0.125864,0.125864,0.125864
8,lgbm,2,1.0,0.137677,NaN,0.137677,0.137677,0.137677,0.137677,0.137677
9,lgbm,3,1.0,0.142550,NaN,0.142550,0.142550,0.142550,0.142550,0.142550


In [13]:
print("RMSE")
display(results_test_df.groupby(["model_meta", "cluster"], as_index=False)["rmse"].describe())

RMSE


,model_meta,cluster,count,mean,std,min,25%,50%,75%,max
0,catboost,0,1.0,0.151648,NaN,0.151648,0.151648,0.151648,0.151648,0.151648
1,catboost,1,1.0,0.163599,NaN,0.163599,0.163599,0.163599,0.163599,0.163599
2,catboost,2,1.0,0.169613,NaN,0.169613,0.169613,0.169613,0.169613,0.169613
3,catboost,3,1.0,0.175440,NaN,0.175440,0.175440,0.175440,0.175440,0.175440
4,catboost,4,1.0,0.173046,NaN,0.173046,0.173046,0.173046,0.173046,0.173046
5,catboost,full,1.0,0.172673,NaN,0.172673,0.172673,0.172673,0.172673,0.172673
6,lgbm,0,1.0,0.153719,NaN,0.153719,0.153719,0.153719,0.153719,0.153719
7,lgbm,1,1.0,0.155509,NaN,0.155509,0.155509,0.155509,0.155509,0.155509
8,lgbm,2,1.0,0.169106,NaN,0.169106,0.169106,0.169106,0.169106,0.169106
9,lgbm,3,1.0,0.177276,NaN,0.177276,0.177276,0.177276,0.177276,0.177276
